<a href="https://colab.research.google.com/github/mkh2097/CI_Text_Recognition/blob/main/Keras_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import os
import re

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU
from keras.preprocessing.text import Tokenizer
from keras.layers.convolutional import Convolution1D
from keras import backend as K

In [ ]:
!pip install hazm
lineList = list()
with open('stop_words.txt') as f:
  for line in f:
    lineList.append(line)

lineList = [x.strip() for x in lineList] 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/train.csv')

In [ ]:
from __future__ import unicode_literals
from hazm import *
import re, string

def remove_stopwords(text):
    text = [word for word in text.split() if word not in lineList]
    return " ".join(text)

normalizer = Normalizer()


special = re.compile(r'\W')
single = re.compile(r'\s+', flags=re.I)

number = re.compile(r"[-+]?[0-9]+")
pnumber = re.compile(r"[-+][\u06F0-\u06F90-9]+")
url = re.compile(r"https?://\S+|www\.\S+")
html = re.compile(r"<.*?>")
emoji_pattern = re.compile(
    "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    "]+",
    flags=re.UNICODE,
)


df["Text"] = df.Text.map(remove_stopwords)
df["Text"] = df.Text.map(lambda x: url.sub(r" ",x))
df["Text"] = df.Text.map(lambda x: html.sub(r" ",x))
df["Text"] = df.Text.map(lambda x: emoji_pattern.sub(r" ",x))
df["Text"] = df.Text.map(lambda x: number.sub(r" ",x))
df["Text"] = df.Text.map(lambda x: pnumber.sub(r" ",x))
df["Text"] = df.Text.map(lambda x: x.translate(str.maketrans(" ", " ", string.punctuation)))
df["Text"] = df.Text.map(lambda x: special.sub(r" ",x))
df["Text"] = df.Text.map(lambda x: single.sub(r" ", x))
df["Text"] = df.Text.map(lambda x: normalizer.normalize(x))


In [ ]:
df.Category = df.Category.astype('category')

nb_classes = df.Category.cat.codes.max() + 1

df["CategoryCodes"] = df.Category.cat.codes


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.Text, df.CategoryCodes, test_size=0.1, random_state=42)

In [ ]:
#best 20000 1000
max_features = 20000
maxlen = 1000

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
# max_features =  len(tokenizer.word_index) + 1
# maxlen =  max([len(s.split()) for s in df.Text])

In [ ]:
X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen, padding='post')
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen, padding='post')


# Y_train = np_utils.to_categorical(y_train, nb_classes)
# Y_test = np_utils.to_categorical(y_test, nb_classes)


print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

X_train shape: (135086, 1000)
X_test shape: (15010, 1000)


In [ ]:
#best = 32
batch_size = 32

In [ ]:
#best 256 512 nodropout
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Embedding(max_features, 300, input_length=maxlen))
model.add(GRU(512, return_sequences=False))
# model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='sigmoid'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


model.fit(X_train, y_train, batch_size=batch_size, epochs=5,
          validation_data=(X_test, y_test), callbacks=callbacks_list)

Epoch 1/5
4222/4222 [==============================] - 2050s 485ms/step - loss: 1.4604 - accuracy: 0.5558 - val_loss: 0.5536 - val_accuracy: 0.8249

Epoch 00001: val_accuracy improved from -inf to 0.82485, saving model to weights-improvement-01-0.82.hdf5
Epoch 2/5
4222/4222 [==============================] - 2049s 485ms/step - loss: 0.4524 - accuracy: 0.8599 - val_loss: 0.5242 - val_accuracy: 0.8353

Epoch 00002: val_accuracy improved from 0.82485 to 0.83531, saving model to weights-improvement-02-0.84.hdf5
Epoch 3/5
4222/4222 [==============================] - 2045s 484ms/step - loss: 0.2855 - accuracy: 0.9118 - val_loss: 0.5573 - val_accuracy: 0.8331

Epoch 00003: val_accuracy did not improve from 0.83531
Epoch 4/5
4222/4222 [==============================] - 2042s 484ms/step - loss: 0.1649 - accuracy: 0.9496 - val_loss: 0.6462 - val_accuracy: 0.8282

Epoch 00004: val_accuracy did not improve from 0.83531
Epoch 5/5
1474/4222 [=========>....................] - ETA: 21:29 - loss: 0.091

KeyboardInterrupt: ignored

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/test.csv.zip')

In [ ]:
df_test["Text"] = df_test.Text.map(remove_stopwords)
df_test["Text"] = df_test.Text.map(lambda x: url.sub(r" ",x))
df_test["Text"] = df_test.Text.map(lambda x: html.sub(r" ",x))
df_test["Text"] = df_test.Text.map(lambda x: emoji_pattern.sub(r" ",x))
df_test["Text"] = df_test.Text.map(lambda x: number.sub(r" ",x))
df_test["Text"] = df_test.Text.map(lambda x: pnumber.sub(r" ",x))
df_test["Text"] = df_test.Text.map(lambda x: x.translate(str.maketrans(" ", " ", string.punctuation)))
df_test["Text"] = df_test.Text.map(lambda x: special.sub(r" ",x))
df_test["Text"] = df_test.Text.map(lambda x: single.sub(r" ", x))
df_test["Text"] = df_test.Text.map(lambda x: normalizer.normalize(x))

In [ ]:
sequences_final = tokenizer.texts_to_sequences(df_test.Text)
X_final = sequence.pad_sequences(sequences_final, maxlen=maxlen, padding='post')
model.load_weights("/content/weights-improvement-02-0.84.hdf5")
Y_final = model.predict(X_final)
Y_classes = Y_final.argmax(axis=-1)

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/train.csv')
# df.Category = df.Category.astype('category')

Y_classes_Name = df.Category.cat.categories[Y_classes]

df_test = df_test.rename(columns = {"Text":"Category"}) 
df_test.Category = Y_classes_Name

df_test.to_csv("Final.csv", index=False)